In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import re

In [2]:
WEB ="https://www.imdb.com/list/ls000634294/"
html_script = requests.get(WEB).text
soup = BeautifulSoup(html_script,'html.parser')

In [3]:
Movie= soup.find("div",class_= "lister-list")
movie_names_list=[]
movie_link_lst=[]
movie_id =[]
review_links=[]
movie_names = Movie.find_all("h3",class_="lister-item-header")
for i in movie_names:
    movie_lst = i.find("a").text
    movie_links = i.find("a")["href"]
    final_links = "https://www.imdb.com"+movie_links+"?ref_=ttls_li_tt"
    revie_links = "https://www.imdb.com"+movie_links+"reviews?ref_=tt_urv"
    movie_id.append(movie_links[9:-1])
    movie_link_lst.append(final_links) 
    movie_names_list.append(movie_lst)
    review_links.append(revie_links)

In [4]:
# if you want to have the movie time and movie genres
# time_list=[]
# genres_list=[]
# for i in movie_link_lst:
#     html_sc = requests.get(i).text
#     info = BeautifulSoup(html_sc,'html.parser')
#     yot = info.find("div","titleBar")
#     time = yot.find("time").text[25:33]
#     genre = yot.find("div","subtext").findAll("a")
#     genres = [g.contents[0] for g in genre]
#     genres = genres[:-1]
#     genres_list.append(genres)
#     time_list.append(time)


In [10]:
test = []
test.append(review_links[0])
test.append(review_links[1])
test

['https://www.imdb.com/title/tt0111161/reviews?ref_=tt_urv',
 'https://www.imdb.com/title/tt0068646/reviews?ref_=tt_urv']

In [12]:
for i in review_links:
    html = requests.get(i).text
    inform = BeautifulSoup(html,"html.parser")
    j = i[29:36]
    z = inform.findAll("div","text show-more__control")
    reviews = [m.text for m in z]
    r = inform.findAll("span","review-date")
    dates = [l.text for l in r]
    rat = inform.findAll("div","lister-item-content")
    ratings=[]
    for ee in rat:
        ra = ee.find("span","rating-other-user-rating")
        if ra==None:
            ratings.append(" ")
        else:
            ratings.append(ra.contents[3].text)
    hel = inform.findAll("div","actions text-muted")
    helpful = []
    helpful_out_of=[]
    for h in hel:
        helpf = h.contents[0]
        helpf = re.sub(r'\s+', ' ', helpf)
        helpf = helpf.split(" ")
        helpful.append(helpf[1])
        helpful_out_of.append(helpf[4])

    uno = inform.find("div","load-more-data")["data-key"]

    while uno!=None:

        page_url = "https://www.imdb.com/title/tt"+j+"/reviews/_ajax?ref_=undefined&paginationKey={}".format(uno)
        code = requests.get(page_url).text
        imp = BeautifulSoup(code,"html.parser")
        k = imp.findAll("div","text show-more__control")
        for s in k:
            reviews.append(s.text)
        t = imp.findAll("span","review-date")
        for d in t:
            dates.append(d.text)
        ot = imp.findAll("div","actions text-muted")
        for ha in ot:
            he = ha.contents[0]
            he = re.sub(r'\s+', ' ', he)
            he = he.split(" ")
            helpful.append(he[1])
            helpful_out_of.append(he[4])
        rats = imp.findAll("div","lister-item-content")
        for aa in rats:
            ro = aa.find("span","rating-other-user-rating")
            if ro==None:
                ratings.append(" ")
            else:
                ratings.append(ro.contents[3].text)
        try :
            uno = imp.find("div","load-more-data")["data-key"]
        except TypeError:
            break

    df = pd.DataFrame({"MovieId":[j]*len(reviews), "Reviews" : reviews , "Date" : dates , "Helpful" : helpful , "Helpful_out_of" : helpful_out_of , "Ratings_out_of_10" : ratings })
    df.to_csv("{}.csv".format(j), index = False)

In [13]:
len(review_links)

100